In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math


from sklearn.tree            import DecisionTreeRegressor
from sklearn.linear_model    import LinearRegression,Ridge,Lasso,RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold,train_test_split 
from sklearn.metrics         import r2_score,mean_squared_error
from sklearn.ensemble        import RandomForestRegressor,  GradientBoostingRegressor 
from xgboost                 import XGBRegressor 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV




C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\train_processedWithLog1pTarget.csv")

In [3]:
test_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\test_processed.csv")

In [4]:
train_df.isnull().sum()

latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
log_(price+1)                        0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [5]:
train_df_with_dummies = train_df

In [6]:
X=train_df_with_dummies.drop(['log_(price+1)'],axis=1)
y=train_df_with_dummies['log_(price+1)']

In [7]:
#y= target
#x = nums
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=24)

In [13]:
model = XGBRegressor(random_state=10)
parameters = {'learning_rate': [0.01,0.02],
                  'n_estimators' : [100],
                  'max_depth'    : [4,6] ,
                  'booster' : ['gbtree'],
                  'sampling_method' : ['uniform','gradient_based'],
                   'subsample' : [0.5,1]
                 }


In [14]:
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 5, n_jobs=2, verbose=9)
grid.fit(X_train, y_train)    

# Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n",grid.best_score_)
    
print("\n The best parameters across ALL searched params:\n",grid.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:   13.1s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:   43.8s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  1.6min
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  2.8min
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:  4.1min
[Parallel(n_jobs=2)]: Done  80 out of  80 | elapsed:  5.0min finished


[10:49:45] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

 Results from Grid Search 

 The best estimator across ALL searched params:
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=10,
             reg_alpha=0, reg_lambda=1, sampling_method='uniform',
             scale_pos_weight=1, seed=None, silent=None, subsample=1,
             verbosity=1)

 The best score across ALL searched params:
 -0.10225111119751332

 The best parameters across ALL searched params:
 {'booster': 'gbtree', 'learning_rate': 0.02, 'max_depth': 6, 'n_estimators': 100, 'sampling_method': 'uniform', 'sub

xgbmodel = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

xgbmodel.fit(X_train,y_train)

In [12]:
y_pred = grid.predict(X_test)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, grid.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, grid.predict(X_test))))

Train RMSE: 0.7203352018199087
Test RMSE: 0.735197058814726


In [15]:
np.sqrt(mean_squared_error(y_train,np.exp(grid.predict(X_train))-1)), np.sqrt(mean_squared_error(y_test,np.exp(grid.predict(X_test))-1))

(72.87906615996123, 72.6893702024274)

## TEST DATA

In [16]:
test_df

id  latitude  longitude  minimum_nights  number_of_reviews  \
0       1525602  40.66751  -73.95867               2                121   
1      30430185  40.75655  -73.99690               1                 18   
2      21354525  40.69252  -73.99121               1                 87   
3      35995074  40.77292  -73.90101               9                  0   
4      34392081  40.71863  -73.94980               1                 28   
...         ...       ...        ...             ...                ...   
14664  27730191  40.59446  -73.94289               2                  1   
14665  29676417  40.74319  -73.98074               2                  1   
14666   1813829  40.82519  -73.94530               1                 93   
14667  25017403  40.72238  -73.99466               1                 15   
14668  19057073  40.68932  -73.94947               5                 23   

       reviews_per_month  calculated_host_listings_count  availability_365  \
0                   1.70                               1                39   
1                   2.49                              30               364   
2                   4.29                               1               108   
3                   0.01                               2               365   
4                  14.00                               2                20   
...                  ...                             ...               ...   
14664               0.59                               1                36   
14665               0.16                               1               365   
14666               1.34                               3               358   
14667               1.12                               8                34   
14668               0.95                               1                 1   

       neighbourhood_group_Bronx  neighbourhood_group_Brooklyn  \
0                              0                             1   
1                              0                             0   
2                              0                             1   
3                              0                             0   
4                              0                             1   
...                          ...                           ...   
14664                          0                             1   
14665                          0                             0   
14666                          0                             0   
14667                          0                             0   
14668                          0                             1   

       neighbourhood_group_Manhattan  neighbourhood_group_Queens  \
0                                  0                           0   
1                                  1                           0   
2                                  0                           0   
3                                  0                           1   
4                                  0                           0   
...                              ...                         ...   
14664                              0                           0   
14665                              1                           0   
14666                              1                           0   
14667                              1                           0   
14668                              0                           0   

       neighbourhood_group_Staten Island  room_type_Entire home/apt  \
0                                      0                          1   
1                                      0                          0   
2                                      0                          0   
3                                      0                          0   
4                                      0                          0   
...                                  ...                        ...   
14664                                  0                          1 

In [17]:
test_id_col = test_df['id']

In [18]:
test_df.isnull().sum()

id                                   0
latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [19]:
test_df_with_dummies = test_df

In [20]:

test_df_with_dummies = test_df_with_dummies.drop(['id'],axis=1)

In [21]:
test_df_with_dummies.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'neighbourhood_group_Bronx',
       'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan',
       'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island',
       'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [22]:
test_df_with_dummies['price']= grid.predict(test_df_with_dummies)

In [23]:
test_df_with_dummies['price'] = np.exp(test_df_with_dummies['price'])-1

In [24]:
submissions_xgbmodel = pd.concat([test_id_col, test_df_with_dummies['price']], axis=1) 


In [25]:
submissions_xgbmodel.to_csv('submission_xgbmodel_gridSearchCV_tune.csv', index=False) 

In [26]:
submissions_xgbmodel

id       price
0       1525602   76.442520
1      30430185   89.710899
2      21354525   52.519958
3      35995074   42.329208
4      34392081   45.816113
...         ...         ...
14664  27730191   66.929901
14665  29676417   63.097000
14666   1813829   43.191944
14667  25017403  109.689758
14668  19057073   68.837090

[14669 rows x 2 columns]

In [27]:
submissions_xgbmodel.isnull().sum()

id       0
price    0
dtype: int64